<a href="https://colab.research.google.com/github/HugoF-Silva/Soft_K_Project/blob/main/v0_0_0002_Soft_K_4_0_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOFT K 4.0
Analysis

# Instruções

1. Crie uma cópia deste arquivo
2. Execute a cópia (clickando no círculo "play" no canto superior esquerdo do código **ou** clickando no código e apertando "ctrl+enter")
3. Leia o item e pense se o item TE REPRESENTA
4. Todo item possibilita 5 opções de resposta:
> Variando de "a" a "e", as opções indicam se você discorda sobre o item te representar, ou se você concorda sobre o item não te representar.
> O 'a' representa "discordo, não me representa".
> O 'e' representa "concordo, me representa".
> As opções entre 'a' e 'e' são níveis moderados de discordância e concordância. 'c' é 'neutro'. Evite respostas 'c'.
5. Evite refletir demais sobre um item (pode distorcer sua resposta)
6. Para a resposta tome como base quem **você é e foi durante os últimos meses** ao invés de *quem você foi anos atrás ou quem você gostaria de ser*.
6. O colab (esta plataforma) pode desconectar durante ao uso do teste, e eventualmente pode ocorrer perda de resultados. Não nos responsabilizamos por isso (o que pode ser uma experiência frustrante, portanto recomendamos responder com internet estável, não interromper o teste enquanto o responde, e salvar o resultado assim que terminar de responder)
7. O resultado não deve ser interpretado como absoluta representação da personalidade de alguém. A ferramenta foi desenvolvida a longo de décadas como uma representação generalista e resumida de características da personalidade. Ainda assim, resultados com este teste foram validados em ampla escala com base em duas teorias com origens distintas e evidências estatísticas.
8. O resultado deve ser considerado "norteador que direciona" o que realmente é a personalidade de alguém, como um suporte que ajuda a entender o comportamento de uma pessoa.
9. O teste leva em torno de 10min a 20min para ser completamente respondido. Aproveite!

In [ ]:
import matplotlib.pyplot as plt
import time 
import random
import zipfile

def print_question(iten, options, label):
    """
    Imprime uma pergunta com as opções disponíveis e retorna a resposta do usuário como um inteiro.
    """
    print(iten)
    letters = ['a', 'b', 'c', 'd', 'e']

    for letter, option in zip(letters, options):
      print(f"{letter} - {option}")
    
    while True:
      answer = input("\n").lower()
      try:
        answer = str(answer)
        # se letra esta nas opções
        if answer in letters:
          # se o nome da escala termina com "pos"
          if label.endswith("pos"):
              # transforma opção em número de escala positiva
              # pega a posição da letra na lista e soma +1
            position = letters.index(answer) + 1
          else:
            # se o nome da escala termina com "neg", pega 6 - (posição da letra na lista + 1)
            position = 6 - (letters.index(answer)+1)
        return position

      except:
        print("Opção inválida, tente novamente.")



#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



def norm0to100(domain_scores, facet_scores, dict_facets):
    """
    Normaliza o resultado do score de domínio para range de 0 a 100 (independente do limite máximo e do limite mínimo de pontuação pro domínio).
    Normaliza o resultado do score de faceta para range de 0 a 25.
    """

    norm_scores = {} 
    lista_scores = [domain_scores, facet_scores]
    for score_type in lista_scores: 
        if score_type == domain_scores:
            facdom = 'norm_domain_scores'
        else:
            facdom = 'norm_facet_scores'
            
        norm_scores[facdom] = {} 

        #exemplo: label -> 'C'  
        for label in score_type: # exemplo de score_type caso seja domain_scores -> {'E': 50, 'N': 48, 'A': 36, 'O': 36, 'C': 48} -> pontuações variadas para cada big trait
                    
            #exemplo:       dict_facets['C']      # dict facets -> {'A': [('A4_neg', 3), ('A1_neg', 0), ('A1_pos', 2), ('A3_pos', 1), ('A3_neg', 1)], 'E': [('E1_pos', 3), etc
            domain_tuples = dict_facets[label[0]] # cada big trait tem sua lista de tuplas as quais representam `(nome de uma escala, quantidade de itens na escala)`
            # se label for C, domain_tuples -> [(C1_neg, 3), (C1_pos, 1), etc]

            # somatório da quantidade de itens medindo uma faceta
            qntd_itens = sum(tuple[1] for tuple in domain_tuples if tuple[0].startswith(label)) # dict facets -> {'A': [('A4_neg', 3), ('A1_neg', 0), ('A1_pos', 2), ('A3_pos', 1), ('A3_neg', 1)], 'E': [('E1_pos', 3), 
            
            if qntd_itens < 1:
                # se qntd_itens for 0, haverá erro na normalização, portanto pulamos a iteração
                continue

            minimum_score = qntd_itens
            maximum_score = qntd_itens * 5

            # normalização  
            if score_type == domain_scores:     
                # normalização para escala 0 a 100 se a iteração atual estiver lidando com score de big trait
                norm_score_tal = ( (score_type[label] - minimum_score) * 100 ) / (maximum_score - minimum_score)
            else:
                # normalização para escala 0 a 25 se a iteração atual estiver lidando com score de faceta
                norm_score_tal = ( (score_type[label] - minimum_score) * 25 ) / (maximum_score - minimum_score)

            # atualiza o sub-dicionário de scores colocando o nome do domínio (ou faceta) e o respectivo resultado da pessoa
            norm_scores[facdom].update({label: norm_score_tal})

    # retorna o sub-dicionários referentes aos scores de domínios normalizados e o sub-dicionário referente aos scores de facetas normalizadas
    return norm_scores['norm_domain_scores'], norm_scores['norm_facet_scores']
          



#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



#Pergunta ao usuário um número de perguntas especificado e retorna a soma das respostas do usuário.
def ask_questions(scales):
    """
    Pega todas as escalas presentes e opera junto da função "print_question" p/ calcular o resultado bruto das respostas de todos os itens de cada escala.
    """
    # para que seja possível futura normalização dos resultados
    # serão adicionados a esse dicionário tuplas com nome da escala e quantidade de itens dela
    dict_label_scales = {}

    # pontuação que será atualizada para cada faceta 
    facet_scores = {'O1': 0, 'O2': 0, 'O3': 0, 'O4': 0, 'O5': 0, 'O6':0,
                    'C1': 0, 'C2': 0, 'C3': 0, 'C4': 0, 'C5': 0, 'C6':0,
                    'E1': 0, 'E2': 0, 'E3': 0, 'E4': 0, 'E5': 0, 'E6':0,
                    'A1': 0, 'A2': 0, 'A3': 0, 'A4': 0, 'A5': 0, 'A6':0,
                    'N1': 0, 'N2': 0, 'N3': 0, 'N4': 0, 'N5': 0, 'N6':0}

    # Copia a lista de escalas para uma nova lista que será randomizada
    random_scales = list(scales)

    # Embaralha a lista nova (objetivo de randomizar os itens para nunca printar na mesma ordem)
    random.shuffle(random_scales)

    # Inicializando variável para armazenar o index da última escala usada
    last_scale_index = -1

    # Inicializando variável para ir adicionando cada item que é respondido (importante para printar apenas itens embaralhados e não usados)
    used_questions = set()

    # nosso limite de iterações é o total de questões na lista de escalas randomizadas
    limit = sum(len(questions) for _, questions, _ in random_scales)
    
    # porcentagens
    list_percent = [0, 8, 16, 25, 33, 42, 50, 59, 67, 76, 84, 92, 100]
    percentage = list_percent[0]
    
    # iteracione printando itens aleatórios (nunca dois itens seguidos de uma mesma escala), atualizando pontuações, criando um dicionário que informa a quantidade de itens para cada escala presente, printando porcentagem de progresso
    # enquanto não tiver usado todas as questões:
    while len(used_questions) < limit:

        # para cada tupla tripla na lista de escalas randômicas
        for scale_label, questions, D in random_scales:
        #Ex.: A3_neg     #len(questions)        #A in random_scales

            # printa a porcentagem do progresso
            if len(used_questions) == ((limit/12) * list_percent.index(percentage)):
                    print(f"{percentage}% CONCLUÍDO")
                    if percentage == 100:
                        break
                    if len(used_questions) < limit:
                        percentage = list_percent[list_percent.index(percentage) + 1]


            # só prossiga se o "conjunto atual de questões" tiver questão pra ser printada
            if questions != []:
                # se a escala atual é a mesma da pergunta anterior, substitua o last_scale_index, e pule essa iteração
                if D == last_scale_index:
                    index_letters = ["O", "C", "E", "A", "N"]
                    if D == index_letters[-1]:
                        last_scale_index = index_letters[0]
                        continue
                    else:
                        last_scale_index = index_letters[index_letters.index(D) + 1]
                        continue

                # lista de itens dentro do conjunto atual de itens em "questions"
                unused_questions = [q for q in questions if q not in used_questions] # mas nessa lista, só tem itens que não estão na lista de used_questions, i.e., lista de questões não-usadas

                if not unused_questions: # se a lista de questões não-usadas estiver vazia:
                    continue
                # senão, pega uma questão da lista de questões não-usadas
                question = random.choice(unused_questions)
                

                # Printa a questão e pega a resposta do usuário
                options = ["Muito impreciso", "Moderadamente impreciso", "Neutro", "Moderadamente preciso", "Muito preciso"]
                position = print_question(question, options, scale_label)
                print("***********************\n\n")

                # Atualiza o score da respectiva faceta
                first_letter = scale_label[0] # First_letter separada da second_letter -> necessário para manipulações futuras relacionadas a apenas first_letter
                second_letter = scale_label[1]
                facet = first_letter + second_letter
                facet_scores[facet] += position

                # Adiciona a questão pra lista de questões usadas
                used_questions.add(question)
                
                # Instancia lista para atualização de dicionário de qntd de itens
                list_of_labels = []
                # se o dicionário de "Qntd de itens" não estiver vazio ----> atualiza o dicionário de acordo com itens que vão sendo respondidos
                if dict_label_scales: # Ex. de dict_label_scales:                {'C': [('C5_pos', 1), ('C5_neg', 3), ('C1_pos', 4)], 'N': [('N5_pos', 2), ('N1_pos', 4)], 'A': [('A4_neg', 4)], 'E': [('E1_pos', 4), ('E2_pos', 2)], ...etc...}
                                      # Ex. de dict_label_scales.values():       dict_values( [ [('N1_pos', 3)], [('O1_pos', 3)] ] )
                                      # Ex. de list(dict_label_scales.values()): [[('N1_pos', 3)], [('O1_pos', 3)]]

                    # for list_of_tuples in [[('N1_pos', 3)], [('O1_pos', 3)]]
                    for list_of_tuples in list(dict_label_scales.values()): # Ex.: [('N1_pos', 3)]

                        # for tuple in [('N1_pos', 3)]
                        for tuple in list_of_tuples: # Ex.: ('N1_pos', 3)      
                            list_of_labels.append(tuple[0]) # Ex.: list_of labels = [] --> ['N1_pos']

                    # só atualiza se não estiver registrado em list_of_labels que a escala já foi usada
                    if scale_label not in list_of_labels:
                        if scale_label[0] not in dict_label_scales.keys():
                            dict_label_scales.update({f"{scale_label[0]}": []})
                        dict_label_scales[scale_label[0]].append((scale_label, len(questions)))

                else: # mas se estiver vazio (primeira iteração)
                    # atualiza o dicionário diretamente sem os loops anteriores
                    dict_label_scales.update({f"{scale_label[0]}": []})
                    dict_label_scales[scale_label[0]].append((scale_label, len(questions)))


                # retira o item respondido da lista de escalas (para que ele não seja printado em próximas iterações)
                if question in random_scales[random_scales.index((scale_label, questions, D))][1]:
                   #random_scales[random_scales.index((scale_label, questions, D))][1] -> exemplo: random_scales[0][1]
                    random_scales[random_scales.index((scale_label, questions, D))][1].remove(question)
                      
                # Atualiza o index da última escala usada
                last_scale_index = D

    
    domain_score_list = {}
    domain_points = 0
    # atualiza um dicionário adicionando resultados de PONTUAÇÕES para cada domínio
    for domain in dict_label_scales: # para cada key (ex.: 'A', 'O', 'C', ..etc..) no dicionario
        # domain points = somatório de valores de uma mesma faceta
        domain_points = sum(facet_scores[facet_label] for facet_label in facet_scores if facet_label.startswith(domain))

        # atualiza o dicionário com a pontuação da pessoa pra cada domínio
        domain_score_list.update({domain: domain_points})
    
    # Normalização de scores (recebe scores para domínio, scores para facetas, e dicionário de quantidade de itens --> necesário para cálculo da normalização)
    norm_domain_scores, norm_facet_scores = norm0to100(domain_score_list, facet_scores, dict_label_scales)
    
    # retorna scores normalizados
    return norm_domain_scores, norm_facet_scores



#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



def generate_scales(prefix, start_index, end_index, suffixes, custom_values={}):
    """
    Cria variáveis representadoras de escalas para armazenar os itens de cada escala com base no dicionário "custom_values"
    """
    scales_names = []
    for i in range(start_index, end_index+1):
        for suffix in suffixes:
            scale_label = f"{prefix}{i}_{suffix}"
            # Exemplo: O1_pos
            # então:   O1_neg
            # então:   O2_pos
            #          etc
            
            dom = prefix #domínio é um big trait
            #             formato da tupla:
            #             append tupla: (nome da escala, itens da escala situada no dicionário, domínio da escala)
            scales_names.append((scale_label, custom_values.get(scale_label, []), dom)) # Ex.:

    return scales_names #Ex.: [('A1_pos', ['Confio nos outros.', 'Tenho fé na boa intenção dos outros.', 'Acredito no que as pessoas dizem.'], 'A'), ('A1_neg', ['Sou receoso com as pessoas.'], 'A'), ('A2_pos', [], 'A'), ('A2_neg', [], 'A'), ..etc..]



#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



def make_graph(domains, facet_results, labels):
    # Separate the facets by their respective domains
    domain_facets = {
        'Abertura': ['O1', 'O3', 'O5'],
        'Conscienciosidade': ['C1', 'C2', 'C4', 'C5'],
        'Extroversão': ['E1', 'E2', 'E3', 'E4'],
        'Afabilidade': ['A1', 'A3', 'A4'],
        'Neuroticismo': ['N1', 'N2', 'N3', 'N5']
    }


    for domain in domains: #Abertura, Conscienciosidade, (...) 
        facets = domain_facets[domain] # domain_facets[Abertura] -> ['O1', 'O3', 'O5']
        # facets -> ['O1', 'O3', 'O5']

        # Get the scores for the current facets
        answered_scores = [facet_results[facet] for facet in facets]
                          # Ex.: facet_scores[O1], facet_scores[O3], facet_scores[O5]

        facet_labels = labels.get(facets[0][0])
        
        print(f"\n\n\n\n======={domain}=======\n")
        for facet_label, answered_score in zip(facet_labels, answered_scores):
            print(f"{facet_label}: {answered_score}")
        print(f"\n")

        # Plot the bar chart for the current domain
        plt.bar(facet_labels, answered_scores)
        plt.xticks(rotation=45, ha='right')
        plt.ylim(0, 29) # o limite de acordo com a normalização é 25, mínimo 0     # o limite original é qntd de itens pra faceta * valor máximo respondível, mínimo é a qntd de itens pra faceta
        plt.title(domain)
        plt.savefig(f"{domain}.png")
        plt.show()



#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



def main():
    print("SOFT K 4.0")

    custom_values = {
        "N1_pos": ["Me preocupo com as coisas.", "Receio sobre o que há de ruim que possa acontecer.", "Tenho medo de muitas coisas", "Me sinto de cabeça cheia facilmente."],

        "N2_pos": ["Fico estressado facilmente.", "Facilmente fico irritado.", "Perco minha calma."],
        "N2_neg": ["Raramente me sinto revoltado."],

        "N3_pos": ["Frequentemente me sinto cabisbaixo.", "Não gosto de mim mesmo.", "Constantemente me sinto deprimido.", "Minhas opiniões voltadas para mim mesmo são desfavoráveis."],

        "N5_pos": ["Normalmente como muito.", "Sacio minhas vontades aos montes."],
        "N5_neg": ["Raramente passo do limite.", "Consigo manter controle sobre minhas compulsões."],



        "E1_pos": ["Faço amizades facilmente.", "Interajo calorosamente com pessoas.", "Me sinto na minha zona de conforto por mais que eu esteja perto de pessoas.", "Ajo com tranquilidade ao interagir com os outros."],

        "E2_pos": ["Adoro festas grandes.", "Converso com muitas pessoas diferentes durante festas."],
        "E2_neg": ["Não gosto de eventos lotados.", "Evito multidões."],

        "E3_pos": ["Assumo comando.", "Induzo-me a conduzir pessoas.", "Assumo controle das coisas."],
        "E3_neg": ["Aguardo a orientação de outros."],

        "E4_pos": ["Estou sempre ocupado.", "Estou sempre transitando de uma coisa pra outra, de um lugar pra outro.", "Faço muita coisa no meu tempo livre.", "Gerencio muitas coisas ao mesmo tempo."],



        "O1_pos": ["Tenho imaginação vívida.", "Curto ir longe com a mente fantasiosamente.", "Gosto passar o dia divagando, sonhar-acordado.", "Gosto de me perder em meio aos pensamentos."],

        "O3_pos": ["Experiencio minhas emoções intensamente."],
        "O3_neg": ["Raramente fico sentimental.", "Meu comportamento não é facilmente afetado pelas minhas emoções.", "Emocionalmente passo por poucos altos e baixos."],

        "O5_neg": ["Não sou interessado em ideias abstratas.", "Espontaneamente evito discussões filosóficas.", "Tenho dificuldade em compreender ideias abstratas.", "Não tenho interesse em discussões teóricas."],



        "A1_pos": ["Confio nos outros.", "Tenho fé na boa intenção dos outros.", "Acredito no que as pessoas dizem."],
        "A1_neg": ["Sou receoso com as pessoas."],

        "A3_pos": ["Me atento aos cuidados com os outros.", "Adoro ajudar os outros."],
        "A3_neg": ["Sou indiferente com os sentimentos dos outros.", "Não tenho tempo para os outros."],

        "A4_neg": ["Amo uma briga boa.", "Elevo o tom com as pessoas.", "Insulto pessoas.", "Dou o troco nos outros."],



        "C1_pos": ["Concluo atividades com sucesso.", "Sobressaio no que eu faço.", "Lido com tarefas tranquilamente.", "Sei como lidar pra fazer as coisas acontecerem."],

        "C2_pos": ["Gosto da ordem (normas).", "Gosto de ajeitar as coisas."],
        "C2_neg": ["Deixo meu quarto bagunçado.", "Deixo largado minhas coisas."],

        "C4_pos": ["Trabalho duro.", "Faço mais do que os outros esperam de mim.", "Estabeleço critérios elevados para mim e para os outros."],
        "C4_neg": ["Não me sinto muito motivado a ter êxito."],

        "C5_pos": ["De imediato dou início aos afazeres."],
        "C5_neg": ["Vejo como trabalhoso agir de fato para pôr os afazares em prática.", "Preciso ser impulsionado pra começar algo.", "Tenho dificuldade em iniciar tarefas."]
    }

    # Gera listas com várias tuplas triplas a partir dos itens inseridos no custom_values
    # cada lista é referente a um domínio
    # cada tupla tipla é referente a faceta de um domínio (seja uma faceta que possui itens no dicionário custom_values ou não)
    # cada faceta tem duas tuplas tiplas, uma pos e outra neg (representação +keyed e -keyed do ipip)
    O_variables = generate_scales("O", 1, 6, ["pos", "neg"], custom_values)
    C_variables = generate_scales("C", 1, 6, ["pos", "neg"], custom_values)
    E_variables = generate_scales("E", 1, 6, ["pos", "neg"], custom_values)
    A_variables = generate_scales("A", 1, 6, ["pos", "neg"], custom_values)
    N_variables = generate_scales("N", 1, 6, ["pos", "neg"], custom_values)

    # junta todas as listas de escalas em uma só
    all_scales = O_variables + C_variables + E_variables + A_variables + N_variables
    

    # Executa mecanismo recomendável de aplicar o questionário
    # Pega a pontação final para cada domínio e pontuação de cada faceta
    domain_score_dict, facet_scores = ask_questions(all_scales)
    

    print(f"\n\n\n\nTOTAL DE PONTOS\n\nExtroversão: {domain_score_dict['E']}\nConscienciosidade: {domain_score_dict['C']}\nNeuroticismo: {domain_score_dict['N']}\nAfabilidade: {domain_score_dict['A']}\nAbertura à experiência: {domain_score_dict['O']}")

    # Gráfico RESULTADO DOS DOMÍNIOS
    x = ['Extroversão', 'Conscienciosidade', 'Neuroticismo', 'Afabilidade', 'Abertura']
    domain_score_list = [domain_score_dict['E'], domain_score_dict['C'], domain_score_dict['N'], domain_score_dict['A'], domain_score_dict['O']]   
    y = domain_score_list

    colors = ['red', 'green', 'blue', 'orange', 'purple']

    # Cria plot
    fig, ax = plt.subplots(figsize=(8,6))
    ax.bar(x, y, color=colors)

    # Ajusta espaçamento entre barras do gráfico
    plt.xticks(rotation=45, ha='right')
    ax.xaxis.set_tick_params(pad=10)

    # limite 100 e mínimo 0 de acordo com a normalização  
    ax.set_ylim([0, 110]) # sem normalização o limite é qntd de itens do big trait * valor máximo respondível, mínimo era a qntd de itens do big trait

    # set label e title
    plt.ylabel('Pontuação')
    plt.title('Resultados do teste de personalidade')
    
    plt.show()


     
    # Gráfico RESULTADO DAS FACETAS
    domain_names = ['Extroversão', 'Conscienciosidade', 'Neuroticismo', 'Afabilidade', 'Abertura']

    facet_names = {'E': ['Simpatia', 'Gregariedade', 'Assertividade', 'Correria'], 
                   'C': ['Autoeficácia', 'Metodismo', 'Busca por conquista', 'Disciplina'], 
                   'N': ['Ansiosidade', 'Raiva', 'Melancolia', 'Excessividade'], 
                   'A': ['Confiança', 'Altruísmo', 'Cooperação'], 
                   'O': ['Imaginação', 'Emotividade', 'Elucidação']
    }

    make_graph(domain_names, facet_scores, facet_names)



# display plot
plt.show()
if __name__ == '__main__':
    main()